# **Importing the Dependinces**

In [1]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:


from google.colab import files
uploaded = files.upload()

Saving movies.csv to movies.csv


In [3]:
#dataset is now stored in a Pandas Dataframe
import io
movie_data=pd.read_csv(io.BytesIO(uploaded['movies.csv']))

In [4]:
#printing the data set
movie_data

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,4798,220000,Action Crime Thriller,NaN,9367,united states\u2013mexico barrier legs arms pa...,es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,...,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,"[{'name': 'Robert Rodriguez', 'gender': 0, 'de...",Robert Rodriguez
4799,4799,9000,Comedy Romance,NaN,72766,NaN,en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,...,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,"[{'name': 'Edward Burns', 'gender': 2, 'depart...",Edward Burns
4800,4800,0,Comedy Drama Romance TV Movie,http://www.hallmarkchannel.com/signedsealeddel...,231617,date love at first sight narration investigati...,en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,...,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,"Signed, Sealed, Delivered",7.0,6,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,"[{'name': 'Carla Hetland', 'gender': 0, 'depar...",Scott Smith
4801,4801,0,NaN,http://shanghaicalling.com/,126186,NaN,en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,...,98.0,"[{""iso_639_1"": ""en

In [5]:
#printig the frist 5 rows of the dataset
movie_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [6]:
#printing the last 5 rows of the dataset
movie_data.tail()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
4798,4798,220000,Action Crime Thriller,NaN,9367,united states\u2013mexico barrier legs arms pa...,es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,...,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,"[{'name': 'Robert Rodriguez', 'gender': 0, 'de...",Robert Rodriguez
4799,4799,9000,Comedy Romance,NaN,72766,NaN,en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,...,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,"[{'name': 'Edward Burns', 'gender': 2, 'depart...",Edward Burns
4800,4800,0,Comedy Drama Romance TV Movie,http://www.hallmarkchannel.com/signedsealeddel...,231617,date love at first sight narration investigati...,en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,...,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,"Signed, Sealed, Delivered",7.0,6,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,"[{'name': 'Carla Hetland', 'gender': 0, 'depar...",Scott Smith
4801,4801,0,NaN,http://shanghaicalling.com/,126186,NaN,en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,...,98.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,A New Yorker in Shanghai,Shanghai Calling,5.7,7,Daniel Henney Eliza Coupe Bill Paxton Alan Ruc...,"[{'name': 'Daniel Hsia', 'gender': 2, 'departm...",Daniel Hsia
4802,4802,0,Documentary,NaN,25975,obsession camcorder crush dream girl,en,My Date with Drew,Ever since the second grade when he first saw ...,1.929883,...,90.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,My Date with Drew,6.3,16,Drew Barrymore Brian Herzlinger Corey Feldman ...,"[{'name': 'Clark Peterson', 'gender': 2, 'depa...",Brian Herzlinger


In [7]:
#check the shape of the dataset
movie_data.shape

(4803, 24)

In [8]:
#getting the more infomation of the dataset
movie_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

In [9]:
#selecting the feature of relevent feature of recommendation 
seleted_features=['genres','keywords','tagline','title','director']

In [10]:
print(seleted_features)

['genres', 'keywords', 'tagline', 'title', 'director']


In [11]:
#replacing the null values with null string
for feature in seleted_features:
    movie_data[feature]=movie_data[feature].fillna(' ')

In [12]:
#combing all the 5 selected features
combined_feature = movie_data['genres']+' '+movie_data['keywords']+' '+movie_data['tagline']+' '+movie_data['title']+' '+movie_data['director']

In [13]:
print(combined_feature)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance   A newlywed couple's honeymoon...
4800    Comedy Drama Romance TV Movie date love at fir...
4801        A New Yorker in Shanghai Shanghai Calling ...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


In [14]:
#converting the text value in numerical values
vectorizer = TfidfVectorizer()

In [15]:
#fitting the TfidfVectorizer
feature_vectors=vectorizer.fit_transform(combined_feature)

In [16]:
print(feature_vectors)

  (0, 1653)	0.25515628427085985
  (0, 5446)	0.18834412689114394
  (0, 774)	0.34027089212212464
  (0, 7636)	0.3134202228436597
  (0, 7418)	0.09299090556617665
  (0, 11596)	0.149542860388702
  (0, 10385)	0.06605852917576412
  (0, 3442)	0.27086291891802733
  (0, 9602)	0.2510748994794632
  (0, 2192)	0.2977135881964922
  (0, 11292)	0.14815923732839528
  (0, 9680)	0.399747221493128
  (0, 4156)	0.1950251250979658
  (0, 2071)	0.26776227834388727
  (0, 2583)	0.25515628427085985
  (0, 3814)	0.12347478999821664
  (0, 9095)	0.12361879509867715
  (0, 3706)	0.13249793601280604
  (0, 255)	0.10745796874979083
  (0, 206)	0.09368413572017883
  (1, 11105)	0.22604756970601722
  (1, 4417)	0.19123077940933303
  (1, 1733)	0.234838042782395
  (1, 7907)	0.22604756970601722
  (1, 1054)	0.1732702481588429
  :	:
  (4800, 8782)	0.10744300189533075
  (4800, 9111)	0.21309451195619789
  (4800, 2212)	0.08127383950727655
  (4800, 3151)	0.06967842343525474
  (4800, 709)	0.19420734394168798
  (4800, 6279)	0.1386925559276

# **COSINE SIMILARITY**

In [17]:
#getting the similaity score using consine
similarity = cosine_similarity(feature_vectors)

In [18]:
print(similarity)

[[1.         0.10850003 0.02226615 ... 0.         0.         0.        ]
 [0.10850003 1.         0.04466461 ... 0.07198464 0.         0.        ]
 [0.02226615 0.04466461 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.07198464 0.         ... 1.         0.         0.06938386]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.06938386 0.         1.        ]]


In [19]:
print(similarity.shape)

(4803, 4803)


In [20]:
#getting the movie name from the error
movie_name = input("Enter your feature movie name: ")

Enter your feature movie name: Avatar


In [21]:
#getting a list movie all movies names given in the dataset
list_of_all_title = movie_data['title'].tolist()

In [22]:
print(list_of_all_title)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [23]:
#finding the close match for the movie name given by the user
find_close_match = difflib.get_close_matches(movie_name,list_of_all_title)

In [24]:

print(find_close_match)

['Avatar']


In [25]:
close_match=find_close_match[0]

In [26]:
print(close_match)

Avatar


In [27]:
index_of_the_movie = movie_data[movie_data.title == close_match]['index'].values[0]

In [28]:
print(index_of_the_movie)

0


In [29]:
#getting a list of similar movies
similarity_score = list(enumerate(similarity[index_of_the_movie]))

In [30]:
print(similarity_score)

[(0, 1.0), (1, 0.1085000265471684), (2, 0.02226614741545797), (3, 0.018085063202458097), (4, 0.15632652954430734), (5, 0.05660437046722024), (6, 0.012099419030122223), (7, 0.05356659330096678), (8, 0.03972562728680751), (9, 0.04191753525799219), (10, 0.10888512849011502), (11, 0.025251606325208915), (12, 0.048026770882992204), (13, 0.02570308207162187), (14, 0.09848367647656367), (15, 0.03880798105115206), (16, 0.051635832721232046), (17, 0.04585471644324074), (18, 0.07436890330731215), (19, 0.058727353191406734), (20, 0.05340090417747348), (21, 0.014647941436467518), (22, 0.042935491586396125), (23, 0.029935833195598106), (24, 0.054146443509279815), (25, 0.09431708557220145), (26, 0.10905819848123746), (27, 0.053794769693932314), (28, 0.07947159811973056), (29, 0.021793139353712538), (30, 0.057920361308512726), (31, 0.08094762977641425), (32, 0.036351564766937516), (33, 0.05278876009587921), (34, 0.0), (35, 0.06432785434396435), (36, 0.06617424813902026), (37, 0.031438265906034005), (

In [32]:
len(similarity_score)

4803

In [33]:
#sorting the movies based on their similaity score
sorted_similarity_movies = sorted(similarity_score, key = lambda x:x[1], reverse=True )


In [34]:
print(sorted_similarity_movies)

[(0, 1.0), (3730, 0.28959880438336116), (1951, 0.28729713854178696), (2403, 0.26577900763482226), (2696, 0.26129066485062685), (643, 0.2557859834857088), (3158, 0.2528527221724024), (661, 0.2508195737914337), (239, 0.2507515256221266), (94, 0.24645152180674346), (4593, 0.24383346415242396), (812, 0.24259090619354098), (1531, 0.23615666422913598), (3439, 0.22878627473084276), (1354, 0.21816903062073775), (461, 0.20785806187506795), (158, 0.19591287932395185), (278, 0.19503883714659864), (1976, 0.19183559810253056), (1650, 0.1840578583866094), (1473, 0.1801199052241842), (4108, 0.17686710329278404), (770, 0.17250369919761221), (3202, 0.1722479115129931), (2198, 0.16505840535836763), (4332, 0.16189143253543292), (539, 0.16013773148000335), (4, 0.15632652954430734), (108, 0.1561452814602582), (150, 0.15600453345091755), (56, 0.15534059426709235), (2900, 0.15529642279968542), (1490, 0.15466014158728195), (305, 0.15407223326512268), (2888, 0.15340796483317087), (1759, 0.1532598022901324), (2

In [35]:
#print the name of similar movies based on the index
print('Movie suggested for you: \n')

i=1
for movie in sorted_similarity_movies:
    index=movie[0]
    title_from_index = movie_data[movie_data.index==index]['title'].values[0]
    if (i<30):
        print(i,'.',title_from_index)
        i+=1

Movie suggested for you: 

1 . Avatar
2 . Cargo
3 . Space Dogs
4 . Aliens
5 . Jason X
6 . Space Cowboys
7 . Alien
8 . Zathura: A Space Adventure
9 . Gravity
10 . Guardians of the Galaxy
11 . Trekkies
12 . Pocahontas
13 . Moonraker
14 . The Terminator
15 . Space Chimps
16 . Lost in Space
17 . Star Trek
18 . Planet of the Apes
19 . Muppets from Space
20 . Wing Commander
21 . The Astronaut's Wife
22 . In the Shadow of the Moon
23 . Event Horizon
24 . Crocodile Dundee
25 . Lockout
26 . Silent Running
27 . Titan A.E.
28 . John Carter
29 . Terminator Genisys


# **Building the Movie Recommender System**

In [36]:
movie_name = input("Enter your feature movie name: ")

Enter your feature movie name: iron man


In [37]:
list_of_all_title = movie_data['title'].tolist()

In [38]:
find_close_match = difflib.get_close_matches(movie_name,list_of_all_title)

In [39]:
close_match=find_close_match[0]

In [40]:
index_of_the_movie = movie_data[movie_data.title == close_match]['index'].values[0]


In [41]:

similarity_score = list(enumerate(similarity[index_of_the_movie]))


In [42]:
sorted_similarity_movies = sorted(similarity_score, key = lambda x:x[1], reverse=True )

In [43]:
#print the name of similar movies based on the index
print('Movie suggested for you: \n')

i=1
for movie in sorted_similarity_movies:
    index=movie[0]
    title_from_index = movie_data[movie_data.index==index]['title'].values[0]
    if (i<11):
        print(i,'.',title_from_index)
        i+=1

Movie suggested for you: 

1 . Iron Man
2 . Iron Man 2
3 . Iron Man 3
4 . Ant-Man
5 . X-Men
6 . Captain America: The Winter Soldier
7 . The Avengers
8 . Made
9 . X-Men: Apocalypse
10 . Avengers: Age of Ultron


# THE END

---




